In [28]:
import requests as reqs
import pandas as pd
import io
import matplotlib.pyplot as plt
from ipywidgets import Dropdown, interact

# Govt website for data pull.
source = 'https://data.london.gov.uk/download/average-house-prices/b1b0079e-698c-4c0b-b8c7-aa6189590ca4/land-registry-house-prices-borough.csv'
source_get = reqs.get(source).content

# Capture data for handling - io used to not require local instance.
data_set = pd.read_csv(io.StringIO(source_get.decode('utf-8')), thousands=',')

# Exclude National Stats.
data_set = data_set[~data_set['Area'].isin(excl_list)]
excl_list = ['North East', 'North West', 'Yorkshire And The Humber', 
             'East Midlands', 'West Midlands', 'East of England', 
             'South East', 'South West', 'England', 'England And Wales', 
             'Wales']
# Adjust years and house values to be integers.
data_set['Year'] = data_set['Year'].str[-4:].astype(int)
data_set['Value'] = data_set['Value'].astype(int)

In [29]:
# UI bits
borough_box=Dropdown(
        options=[x for x in data_set['Area'].unique()],
        value='London',
        description='Pick Area:',
        layout={'width': 'max-content'},
        style = {'description_width': 'initial'},
        disabled=False)
data_box=Dropdown(
        options=['Value', 'Val Chg YoY'],
        value='Value',
        description='Plot Value:',
        layout={'width': 'max-content'},
        style = {'description_width': 'initial'},
        disabled=False)
year_box=Dropdown(
        options=[x for x in data_set['Year'].unique()],
        value=2010,
        description='Min Year:',
        layout={'width': 'max-content'},
        style = {'description_width': 'initial'},
        disabled=False)

In [30]:
@interact(borough = borough_box, pull_val = data_box, min_year = year_box)
def init_filter(borough, pull_val, min_year):
    # Only capture market median observations.
    data_adj = data_set.loc[data_set['Measure'] == 'Median']
    
    # Pick min timeline and area.
    data_adj = data_adj.loc[data_adj['Area'] == borough]
    data_adj = data_adj.loc[data_adj['Year'] >= min_year]
    
    # Add change in prices, to see relative movement of the market.
    data_adj['Val Chg YoY'] = data_adj['Value'].diff()
    
    # Create timeline of value chosen and year - keep 'data_adj' set.
    data_final = data_adj.groupby(['Year'])[pull_val].mean()
    data_final.plot.line(x='Year',y=pull_val)
    return

interactive(children=(Dropdown(description='Pick Area:', index=33, layout=Layout(width='max-content'), options…